In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
stVLE = pd.read_csv('studentVle.csv')

In [3]:
# only use the data from the course start
stVLE_update = stVLE[stVLE['date']>=0]

In [4]:
stAss = pd.read_csv('studentAssessment.csv')
stReg = pd.read_csv('studentRegistration.csv')
stInfo = pd.read_csv('studentInfo.csv')
VLE = pd.read_csv('vle.csv')

In [5]:
# extract pass, disctinctio and fail students
stInfo_update = stInfo[(stInfo['final_result'] =='Pass') | (stInfo['final_result'] =='Distinction')
                       | (stInfo['final_result'] =='Fail')] 

In [6]:
courses = stInfo_update['code_module'].unique()
dic_site_acitivity = {v:k for v, k in zip(VLE['id_site'], VLE['activity_type'])}
actions_len = len(VLE['activity_type'].unique())

actions_dic = VLE['activity_type'].unique().tolist()

actions_dic = {v: k for k, v in enumerate(actions_dic)}

dic_acitivity_idx = {v:k for v, k in actions_dic.items()}

print(dic_acitivity_idx)

{'resource': 0, 'oucontent': 1, 'url': 2, 'homepage': 3, 'subpage': 4, 'glossary': 5, 'forumng': 6, 'oucollaborate': 7, 'dataplus': 8, 'quiz': 9, 'ouelluminate': 10, 'sharedsubpage': 11, 'questionnaire': 12, 'page': 13, 'externalquiz': 14, 'ouwiki': 15, 'dualpane': 16, 'repeatactivity': 17, 'folder': 18, 'htmlactivity': 19}


In [ ]:
# extract features for the course FFF, 2013 as the training and 2014 as the test

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

labelencoder.fit(["Distinction", "Fail", "Pass"])

courses = ['FFF'] # we can also add AAA, BBB, CCC

for i, course in enumerate(courses):
    
    stVLE_subset = stVLE_update[stVLE_update['code_module'] == course]
    stInfo_update_filtered = stInfo_update[stInfo_update['code_module'] == course]
    
    # filter multiple attemps
    stInfo_update_filtered.drop_duplicates(subset=['id_student'], keep=False, inplace=True)
    
    stVLE_subset_filtered = stVLE_subset[stVLE_subset['id_student'].isin(stInfo_update_filtered['id_student'])]
    
    stInfo_update_filtered = stInfo_update_filtered[stInfo_update_filtered['id_student'].isin(stVLE_subset_filtered['id_student'])]
    
    
    stVLE_subset_train = stVLE_subset_filtered[(stVLE_subset_filtered['code_presentation'] == '2013B') | 
                                              (stVLE_subset_filtered['code_presentation'] == '2013J')]

    stVLE_subset_test = stVLE_subset_filtered[(stVLE_subset_filtered['code_presentation'] == '2014B') | 
                                              (stVLE_subset_filtered['code_presentation'] == '2014J')]
  
    stinfo_train = stInfo_update_filtered[stInfo_update_filtered['id_student'].isin(stVLE_subset_train['id_student'])]

    stinfo_test = stInfo_update_filtered[stInfo_update_filtered['id_student'].isin(stVLE_subset_test['id_student'])]

    
    studentno_train = len(stVLE_subset_train['id_student'].unique())
    
     
    stinfo_train['arrayIdx'] = np.arange(studentno_train)
    
     
    print('the number of students {} in train course {}'.format(studentno_train, course))
    
    cc = stVLE_subset_train.replace({'id_site': dic_site_acitivity}, inplace=False) 
    stVLE_subset_train['activity_type'] = cc['id_site']
    
    cc = stVLE_subset_train.replace({'activity_type': dic_acitivity_idx}, inplace=False) 
    stVLE_subset_train['activity_idx'] = cc['activity_type']
   
    
    stVLE_subset_train = pd.merge(stVLE_subset_train, stinfo_train[{'arrayIdx', 'id_student'}], on='id_student', validate='m:1')
    
    total_days = np.max(stVLE_subset_train['date'].values) + 1
    
    actionArray_train = np.zeros([studentno_train, total_days, actions_len])
    
    y_train = np.zeros(studentno_train)
         
    y_train = labelencoder.transform(stinfo_train['final_result'])
   

    for day in range(total_days):
        
        #print('running the course {} and the day : {}'.format(course, day))
        
        stVLE_days = stVLE_subset_train[stVLE_subset_train['date'] == day]
    
        yy = stVLE_days.groupby(['arrayIdx', 'activity_idx'])['sum_click'].sum().unstack(fill_value=0)

        newyy = yy.reindex(np.arange(actions_len), axis=1, fill_value=0)

        newzz = newyy.reindex(np.arange(studentno_train), axis=0, fill_value=0)

        newzz = newzz.values

        actionArray_train[:,day,:] = newzz
        
    ####
    
    studentno_test = len(stVLE_subset_test['id_student'].unique())
    
     
    stinfo_test['arrayIdx'] = np.arange(studentno_test)
    
     
    print('the number of students {} in test course {}'.format(studentno_test, course))
    
    cc = stVLE_subset_test.replace({'id_site': dic_site_acitivity}, inplace=False) 
    stVLE_subset_test['activity_type'] = cc['id_site']
    
    cc = stVLE_subset_test.replace({'activity_type': dic_acitivity_idx}, inplace=False) 
    stVLE_subset_test['activity_idx'] = cc['activity_type']
   
    
    stVLE_subset_test = pd.merge(stVLE_subset_test, stinfo_test[{'arrayIdx', 'id_student'}], on='id_student', validate='m:1')
    
    total_days = np.max(stVLE_subset_test['date'].values) + 1
    
    actionArray_test= np.zeros([studentno_test, total_days, actions_len])
    
    y_test = np.zeros(studentno_test)
         
    y_test = labelencoder.transform(stinfo_test['final_result'])
   
    
    for day in range(total_days):
        
        #print('running the course {} and the day : {}'.format(course, day))
        
        stVLE_days = stVLE_subset_test[stVLE_subset_test['date'] == day]
    
        yy = stVLE_days.groupby(['arrayIdx', 'activity_idx'])['sum_click'].sum().unstack(fill_value=0)

        newyy = yy.reindex(np.arange(actions_len), axis=1, fill_value=0)

        newzz = newyy.reindex(np.arange(studentno_test), axis=0, fill_value=0)

        newzz = newzz.values

        actionArray_test[:,day,:] = newzz
 

In [ ]:
# import pickle
# with open("saved_click_data_FFF.pkl", "wb") as fp:   #Pickling
#     pickle.dump(savedata, fp)

In [8]:
# merge Distinction and pass
y_train[y_train==2] = 0
y_test[y_test==2] = 0

In [9]:
# print the number of positive and negative samples

[a, b] = np.unique(y_train, return_counts=True)
print(a)
print(b)

[a, b] = np.unique(y_test, return_counts=True)
print(a)
print(b)

[0 1]
[1918  952]
[0 1]
[1759  742]


In [11]:
total_weeks = 37

usedweeks = 10  # the weeks' data we used for prediction, e.g 5, 10, 15, 35 etc

X = list()

for i in range(total_weeks):
    seq_x = np.sum(actionArray_train[:, (7*i):(7*i+6), :], axis=1)
    X.append(seq_x)
X = np.array(X)

actionWeeks = np.swapaxes(X, 0, 1)

def prepare_data(actionWeeks, weeks=1):
    return actionWeeks[:,:weeks,:]

X_train = prepare_data(actionWeeks, weeks=usedweeks)


In [12]:
# scale the training and test data by the log transformation

X = list()

for i in range(total_weeks):
    seq_x = np.sum(actionArray_test[:, (7*i):(7*i+6), :], axis=1)
    X.append(seq_x)
X = np.array(X)

actionWeeks = np.swapaxes(X, 0, 1)

def prepare_data(actionWeeks, weeks=1):
    return actionWeeks[:,:weeks,:]

X_test = prepare_data(actionWeeks, weeks=usedweeks)

from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

XX = np.reshape(X_train, (X_train.shape[0]*X_train.shape[1], X_train.shape[2]))
scaler.fit(np.log(XX + 1))
XX = scaler.transform(np.log(XX+1))

X_train = np.reshape(XX, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

XX_test = np.reshape(X_test, (X_test.shape[0]*X_test.shape[1], X_test.shape[2]))
XX_test = scaler.transform(np.log(XX_test+1))
X_test = np.reshape(XX_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))


In [13]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Bidirectional, LayerNormalization
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import regularizers
import re
from keras.layers import Dropout

In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

# METRICS = [
#       keras.metrics.TruePositives(name='tp'),
#       keras.metrics.FalsePositives(name='fp'),
#       keras.metrics.TrueNegatives(name='tn'),
#       keras.metrics.FalseNegatives(name='fn'), 
#       keras.metrics.BinaryAccuracy(name='accuracy'),
#       keras.metrics.Precision(name='precision'),
#       keras.metrics.Recall(name='recall'),
#       keras.metrics.AUC(name='auc'),
# ]

METRICS = [
    #keras.metrics.TruePositives(name='tp'),
    keras.metrics.AUC(name='accuracy'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.AUC(name='auc')
]


n_steps = X_train.shape[1]
n_features = X_train.shape[2]



In [15]:

tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

from tensorflow.keras import initializers

model = Sequential()
model.add(tf.keras.layers.Masking(mask_value=0.,
                                   input_shape=(n_steps, n_features))) # skip the weeks without data
model.add(LSTM(300, return_sequences=False, 
               dropout=0.1, recurrent_dropout=0.1))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.01),metrics = METRICS)
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 10, 20)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 300)               385200    
_________________________________________________________________
dense (Dense)                (None, 1)                 301       
Total params: 385,501
Trainable params: 385,501
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
batch_size = 100
my_callbacks = [early_stopping]

neg = len(y_train[y_train ==0])
pos = len(y_train[y_train ==1])
total = neg + pos
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

history = model.fit(X_train, y_train, epochs = 100, batch_size=batch_size, 
                    validation_data=(X_test, y_test), 
                    callbacks = my_callbacks, class_weight=class_weight,
                    verbose = 2, shuffle=True)

Epoch 1/100
29/29 - 2s - loss: 0.6566 - accuracy: 0.6884 - recall: 0.6040 - precision: 0.5057 - auc: 0.6884 - val_loss: 0.5823 - val_accuracy: 0.7695 - val_recall: 0.7035 - val_precision: 0.5283 - val_auc: 0.7695
Epoch 2/100
29/29 - 2s - loss: 0.6147 - accuracy: 0.7397 - recall: 0.6901 - precision: 0.5281 - auc: 0.7397 - val_loss: 0.6275 - val_accuracy: 0.7945 - val_recall: 0.8315 - val_precision: 0.4646 - val_auc: 0.7945
Epoch 3/100
29/29 - 2s - loss: 0.5779 - accuracy: 0.7750 - recall: 0.6891 - precision: 0.5739 - auc: 0.7750 - val_loss: 0.5796 - val_accuracy: 0.8027 - val_recall: 0.7911 - val_precision: 0.4949 - val_auc: 0.8027
Epoch 4/100
29/29 - 1s - loss: 0.5624 - accuracy: 0.7848 - recall: 0.7258 - precision: 0.5524 - auc: 0.7848 - val_loss: 0.4878 - val_accuracy: 0.8121 - val_recall: 0.5714 - val_precision: 0.6347 - val_auc: 0.8121
Epoch 5/100
29/29 - 2s - loss: 0.5535 - accuracy: 0.7916 - recall: 0.7164 - precision: 0.5641 - auc: 0.7916 - val_loss: 0.6087 - val_accuracy: 0.809

In [17]:
results = model.evaluate(X_test, y_test)

79/79 [==============================] - 1s 9ms/step - loss: 0.4878 - accuracy: 0.8121 - recall: 0.5714 - precision: 0.6347 - auc: 0.8121


In [ ]:
plot_history(history, model)


%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

y_pred_train=model.predict_classes(X_train)
con_mat = tf.math.confusion_matrix(labels=y_train, predictions=y_pred_train).numpy()

cm_plot_labels = ['safe','risky']
cm = confusion_matrix(y_true=y_train, y_pred=y_pred_train)


plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='training Confusion Matrix')


y_pred_test=model.predict_classes(X_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred_test).numpy()

cm_plot_labels = ['safe','risky']
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_test)


plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='test Confusion Matrix')

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history, model):
    metrics = model.metrics_names
    plt.figure(figsize=(12, 8))
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        
        plt.subplot(3,2,n+1)
        plt.plot(history.epoch,  history.history[metric], color='b', label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
             color='r', linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        plt.legend()
    plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')